In [1]:
import cv2, pywt, numpy as np
from pathlib import Path

In [7]:
# ========== 1. Load host & watermark ==========
host_path      = Path(r"images\chihuahua.webp")        # any 8-bit gray image, e.g. 512×512
watermark_path = Path(r"images\watermark.jpg")          # a *smaller* gray logo, e.g. 64×64
host_img       = cv2.imread(str(host_path), cv2.IMREAD_GRAYSCALE)
wm_img         = cv2.imread(str(watermark_path), cv2.IMREAD_GRAYSCALE)
if host_img is None or wm_img is None:
    raise FileNotFoundError("Check host / wm paths")

# optional: make watermark square & power-of-two for nicer shapes
host_img = cv2.resize(host_img, (512, 512), interpolation=cv2.INTER_AREA)
wm_img = cv2.resize(wm_img, (64, 64), interpolation=cv2.INTER_AREA)

cv2.imshow("Host", host_img)
cv2.imshow("Watermark", wm_img)
cv2.waitKey(0)

-1

In [3]:
# ========== 2. Forward DWTs ==========
wavelet, host_level = "haar", 4
wm_level = 1
alpha = 0.04

coeffs_host = pywt.wavedec2(host_img, wavelet=wavelet, level=host_level)
cA4, detail_host   = coeffs_host[0], coeffs_host[1:]        # host coeff stack

coeffs_wm  = pywt.wavedec2(wm_img,  wavelet=wavelet, level=wm_level)
cA_wm, (cH_wm, cV_wm, cD_wm) = coeffs_wm                   # watermark LL₁ + details

print(f"Fused cV4 shape: {cA4.shape}, Watermark shape: {cA_wm.shape}")

Fused cV4 shape: (32, 32), Watermark shape: (32, 32)


### DWT coefficient fusion

In [4]:
# ========== 3. Coefficient-fusion rule ==========
# We embed watermark’s LL₁ into host’s HL₄ (cV4) after resizing if shapes differ.
cH4, cV4, cD4           = detail_host[0]                   # HL₄, etc.
cH4_wm                  = cH4 + alpha * cA_wm          # fusion

# put back the modified sub-band
detail_host[0]         = (cH4_wm, cV4, cD4)
coeff_emb            = [cA4] + detail_host

# print(f"Fused cV4 shape: {cV4_wm.shape}, Watermark shape: {cA_wm.shape}")

### with SVD

In [92]:
# ========== 3. SVDs ==========
cH4, cV4, cD4           = detail_host[0]   
U_h, S_h, Vh_h      = np.linalg.svd(cV4, full_matrices=False)
U_w,   S_w, Vh_w         = np.linalg.svd(cA_wm, full_matrices=False)

S_emb               = S_h + alpha * S_w
cV4_emb             = (U_h @ np.diag(S_emb) @ Vh_h)

# put back & inverse DWT
detail_host[0]     = (cH4, cV4_emb, cD4)
coeff_emb           = [cA4] + detail_host

In [6]:
# ========== 4. Inverse DWT ⇒ watermarked image ==========
watermarked_img = (
    pywt.waverec2(coeff_emb, wavelet)
    .clip(0, 255)
    .astype(np.uint8)
)
cv2.imwrite("watermarked.png", watermarked_img)
print("✅ Watermarked image saved → watermarked.png")

cv2.imshow("Watermarked Image", watermarked_img)
cv2.waitKey(0)

✅ Watermarked image saved → watermarked.png


-1

In [94]:
# ========== 5. PSNR (cover ↔ watermarked) ==========
def psnr(a, b, max_val=255.0):
    mse = np.mean((a.astype(np.float32) - b.astype(np.float32)) ** 2)
    return 20 * np.log10(max_val) - 10 * np.log10(mse)

print(f"PSNR: {psnr(host_img, watermarked_img):.2f} dB")

PSNR: 46.74 dB


### Extraction (DWT coefficient fusion)

In [95]:
# # ========== 6. Extract watermark & BER ==========
# # (In practice, start from an attacked/received image.)
# wmd_path      = Path(r"watermarked.png")
# wmd_img         = cv2.imread(str(wmd_path), cv2.IMREAD_GRAYSCALE)

# if wmd_img is None:
#     raise FileNotFoundError("Check wmd path")

# wmd_img = cv2.resize(wmd_img, (512, 512), interpolation=cv2.INTER_AREA)

# coeffs_rcv            = pywt.wavedec2(wmd_img, wavelet=wavelet, level=host_level)
# cA4_rcv, det_rcv      = coeffs_rcv[0], coeffs_rcv[1:]
# cH4_rcv, cV4_rcv, _   = det_rcv[0]

# extracted_LL1         = (cV4_rcv - cV4) / alpha # reverse the fusion
# extracted_LL1         = np.round(extracted_LL1).astype(np.int16)

# # Optionally rebuild the watermark image for visual inspection
# coeffs_wm_rcv  = [extracted_LL1, (np.zeros_like(cH_wm),
#                                   np.zeros_like(cV_wm),
#                                   np.zeros_like(cD_wm))]
# wm_rcv_img     = pywt.waverec2(coeffs_wm_rcv, wavelet).clip(0,255).astype(np.uint8)
# cv2.imwrite("wm_extracted.png", wm_rcv_img)
# print("✅ Extracted watermark saved → wm_extracted.png")

# cv2.imshow("Extracted Watermark", wm_rcv_img)
# cv2.waitKey(0)

### Extraction (SVD)

In [99]:
# ========== 6. Extract watermark & BER ==========
# (In practice, start from an attacked/received image.)
wmd_path      = Path(r"wa_watermarked.jpg")
wmd_img         = cv2.imread(str(wmd_path), cv2.IMREAD_GRAYSCALE)

if wmd_img is None:
    raise FileNotFoundError("Check wmd path")

wmd_img = cv2.resize(wmd_img, (512, 512), interpolation=cv2.INTER_AREA)

coeffs_rcv            = pywt.wavedec2(wmd_img, wavelet=wavelet, level=host_level)
cA4_rcv, det_rcv      = coeffs_rcv[0], coeffs_rcv[1:]
cH4_rcv, cV4_rcv, _   = det_rcv[0]

# ---- 2. SVD on the received sub-band ----
U_r, S_r, Vh_r   = np.linalg.svd(cV4_rcv, full_matrices=False)

# ---- 3. Recover watermark Σ ----
S_w_rec          = (S_r - S_h) / alpha

# ---- 4. Rebuild LL₁ estimate of watermark ----
wm_LL_est        = U_w @ np.diag(S_w_rec) @ Vh_w        # uses original U_h & V_h
wm_rcv_img           = pywt.waverec2([wm_LL_est, (np.zeros_like(wm_LL_est),
                                             np.zeros_like(wm_LL_est),
                                             np.zeros_like(wm_LL_est))],
                                wavelet).clip(0,255).astype(np.uint8)

cv2.imwrite("wm_extracted.png", wm_rcv_img)
print("✅ Extracted watermark saved → wm_extracted.png")

cv2.imshow("Extracted Watermark", wm_rcv_img)
cv2.waitKey(0)

✅ Extracted watermark saved → wm_extracted.png


-1

In [100]:
# ---- 5. Metrics ----
orig_bits = (wm_img > 127).astype(np.uint8)
recv_bits = (wm_rcv_img[:wm_img.shape[0], :wm_img.shape[1]] > 127).astype(np.uint8)
ber       = np.mean(orig_bits != recv_bits)
print(f"BER  : {ber:.4f}")
print(f"PSNR : {psnr(host_img, wmd_img):.2f} dB (cover ↔ watermarked)")

BER  : 0.0876
PSNR : 44.45 dB (cover ↔ watermarked)


In [98]:
# def calculate_ber(transmitted_bits, received_bits):
#     """
#     Calculates the Bit Error Rate (BER) between two bit sequences.

#     Args:
#         transmitted_bits (list or array-like): The sequence of bits that were transmitted.
#         received_bits (list or array-like): The sequence of bits that were received.

#     Returns:
#         float: The calculated Bit Error Rate (BER).
#               Returns 0.0 if the sequences are empty or if no errors occur.
#               Returns None if the lengths of the sequences do not match.
#     """
#     # Convert to numpy arrays for easier handling
#     transmitted_bits = np.array(transmitted_bits)
#     received_bits = np.array(received_bits)
    
#     if len(transmitted_bits) != len(received_bits):
#         print("Error: Transmitted and received bit sequences must have the same length.")
#         return None

#     if len(transmitted_bits) == 0:  # Handle empty sequences
#         return 0.0

#     # Calculate errors using vectorized operations
#     errors = np.sum(transmitted_bits != received_bits)
#     ber = errors / len(transmitted_bits)
#     return ber

# # Convert to binary for proper BER calculation
# def to_binary_array(img_array, threshold=128):
#     """Convert grayscale values to binary (0/1)"""
#     return (img_array > threshold).astype(np.uint8)

# # Calculate BER with binary conversion
# original_bits = to_binary_array(cA_wm)
# extracted_bits = to_binary_array(extracted_LL1)

# ber = calculate_ber(original_bits.flatten(), extracted_bits.flatten())
# print(f"BER:  {ber:.4f}")

# # Also calculate correlation for additional metric
# correlation = np.corrcoef(cA_wm.flatten(), extracted_LL1.flatten())[0,1]
# print(f"Correlation: {correlation:.4f}")